
# Gathering information about a server from the Windows Event logs for a particular time

Jess Pomfret has written a great blog about how you can use PowerShell to gather the Windows Event logs from a server for a particular period of time

[https://jesspomfret.com/get-winevent/](https://jesspomfret.com/get-winevent/)

## Choosing the logs that you want to investigate

You might choose to just start with the system and application event logs, in which case, skip the next block, but if you wish to dig further, you will need to know the names of the logs. You can find the list of Windows Event Logs with records on a server with the code below. It will pop-up in a seperate window which will probably be hidden but you will see the flashing PowerShell Icon in the taskbar.

You can search in the out-gridview box in the top bar to filter. It will take a couple minutes to run.



In [ ]:
$computerName = ''
Get-WinEvent -ListLog * -ComputerName $computerName  | 
Where-Object RecordCount | 
Out-GridView

Once you have decided which logs to interogate , you can add the computer name, the date and time you wish to investigate and the window in minutes. This will search before and after the time, you specify

In [ ]:
$computerName = ''   ## Add the computername here
$issueDateTime = get-date('2020-07-31 21:30')   ## Add the date and time you are interested in here in the format 'yyyy-MM-dd HH:mm'
$windowMins = 30  ## The number of minutes before and after the issue date time to gather information for
$lognames = 'system','application' ## The log names you wish to interogate

$winEventFilterHash = @{
    LogName = $lognames
    StartTime = $issueDateTime.AddMinutes(-($windowMins/2))
    EndTime = $issueDateTime.AddMinutes(($windowMins/2))
}

# Run this first to make sure output width does not mess with output - Update output buffer size to prevent clipping in Visual Studio Code output window.
if( $Host -and $Host.UI -and $Host.UI.RawUI ) {
    $rawUI = $Host.UI.RawUI
    $oldSize = $rawUI.BufferSize
    $typeName = $oldSize.GetType( ).FullName
    $newSize = New-Object $typeName (500, $oldSize.Height)
    $rawUI.BufferSize = $newSize
  }

Get-WinEvent -FilterHashtable $winEventFilterHash -ComputerName $computerName | Select-Object LogName,
ProviderName,
TimeCreated,
Id,
LevelDisplayName,
@{l='UserName';e={(New-Object System.Security.Principal.SecurityIdentifier($_.UserId)).Translate([System.Security.Principal.NTAccount])}}, 
Message | Format-Table -AutoSize -Wrap

## Exporting it to Excel

If you wish to export the results to Excel for analysis, you can use the ImportExcel module and the code below. Again, you will need to update the Computer name, Windows Event Log names and time as before

In [ ]:
$computerName = ''   ## Add the computername here
$issueDateTime = get-date('2020-07-31 21:30')   ## Add the date and time you are interested in here in the format 'yyyy-MM-dd HH:mm'
$windowMins = 30  ## The number of minutes before and after the issue date time to gather information for
$lognames = 'system','application'  ## The log names you wish to interogate
$filepath = 'C:\temp\'    # path to folder for the Excel workbook

$winEventFilterHash = @{
    LogName = 'system','application'
    StartTime = $issueDateTime.AddMinutes(-($windowMins/2))
    EndTime = $issueDateTime.AddMinutes(($windowMins/2))
}

$issueDateTimeexcel = Get-Date($issueDateTime) -Format 'yyyy-MM-dd_HH_MM'

$Worksheetname = $computerName.split('.')[0] + '_' + $issueDateTimeexcel
$filepath = $filepath + $WorksheetName + '.xlsx'
Get-WinEvent -FilterHashtable $winEventFilterHash -ComputerName $computerName | Select-Object LogName,
ProviderName,
TimeCreated,
Id,
LevelDisplayName,
@{l='UserName';e={(New-Object System.Security.Principal.SecurityIdentifier($_.UserId)).Translate([System.Security.Principal.NTAccount])}}, 
Message | Export-Excel -Path $filepath -FreezeTopRow -AutoSize -AutoFilter -WorksheetName $Worksheetname -BoldTopRow 

Invoke-Item $filepath